# Clustering and Modelling

>1. Topic distribution of each patient using LDA <br>
>2. cluster the top distribution using clustering algorithm <br>
>3. Model each cluster <br>
>4. Build ROC Curve <br>
>5. Confusion matrix <br>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Load Data

In [2]:
"""
 df: Feature dataframe
 dfNotes: Notes dataframe
"""
df = pd.read_csv('notebooks/cse6250_features.csv')
dfNotes = pd.read_csv('topicfeatures.csv',header=None)
dfNotes.columns = ['subject_id'] + ['topic_'+str(x) for x in range(1,21)]


In [3]:
from sklearn.cluster import KMeans, SpectralClustering

n_clusters = 4

## KMeans Algo

In [6]:
"""
KMeans Algo find clusters
Merge features and cluster
"""
kmeans = KMeans(n_clusters=n_clusters).fit(dfNotes[dfNotes.columns[1:]])
dfNotes['cluster'] = kmeans.labels_


In [87]:
finalOutput = {}

## Model each cluster

In [11]:
import os
import sys
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim

from utils import xtrain, evaluate
from mymodels import loadDataSet,MyCNN, MyRNN
from stats import Model,statScores,func,splitNRunModel
from plots import plot_learning_curves, plot_confusion_matrix

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from xgboost import plot_importance

In [49]:
def rocCurve(df,model):
    y_test,y_pred,y_prob = df.y_test,df.y_pred,df.y_prob
    logit_roc_auc = roc_auc_score(y_test, y_pred)
    fpr, tpr, thresholds = roc_curve(y_test, y_prob)
    plt.plot(fpr, tpr, label= model + ' (area = %0.2f)' % logit_roc_auc)
    print(logit_roc_auc)
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")
#     plt.savefig(saveFig)
    plt.show()

In [50]:
# features = [feature for feature in data.columns if feature not in 
#             ['angus','subject_id','cluster','dob','gender','age_group','maxDate','first_admittime']]
# # features

In [29]:
def runNewModel(train,test,mlModel):
    global features
    X_train,y_train = train.loc[:, features],train.angus
    X_test,y_test = test.loc[:, features],test.angus
    model = mlModel.model
    model.fit(X_train, y_train)
    if('xgboost'==mlModel.label):
        plot_importance(model,max_num_features=15)
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:,1]
    df = pd.DataFrame({'y_test':y_test,'y_pred':y_pred,'y_prob':y_prob})
    return df

In [53]:
import torch.nn.functional as F
class MyCNN(nn.Module):
	def __init__(self,size):
		super(MyCNN, self).__init__()
		self.size = size
		self.conv1 = nn.Conv1d(in_channels=1,out_channels=6,kernel_size=5)
		self.pool = nn.MaxPool1d(kernel_size=2)
		self.conv2 = nn.Conv1d(in_channels=6,out_channels=16,kernel_size=5)
		self.linear1 = nn.Linear(self.size,128)
		self.linear2 = nn.Linear(128,64)
		self.linear_out = nn.Linear(128,2)

	def forward(self, x):
		x = self.pool(F.relu(self.conv1(x)))
		x = self.pool(F.relu(self.conv2(x)))
		# print(x.shape)
		x = x.view(-1, self.size)
		x = F.relu(self.linear1(x))
		x = self.linear_out(x)
		return x

In [77]:
result = []
models = []
PATH_OUTPUT = "./mimic/sepsis/final/"
os.makedirs(PATH_OUTPUT, exist_ok=True)

In [84]:


def runCNNRNN(cols,label,trainDF,testDF,PATH_OUTPUT):
    BATCH_SIZE = 42
    size   = 16*21
    
    os.makedirs(PATH_OUTPUT, exist_ok=True)
    output_dict = {}
        
    if(label == '_with_Out_Notes'):
        BATCH_SIZE = 32
        size = 16*16
    for modelType in ['CNN','RNN']:
        if modelType=='RNN':
            model = MyRNN()
            save_file = 'myRNN'+label+'.pth'

        elif modelType == 'CNN':
            model = MyCNN(size)
            save_file = 'myCNN'+label+'.pth'
#             BATCH_SIZE = 42

#         models.append(modelType+label)
        NUM_EPOCHS = 10	
        USE_CUDA = False  # Set 'True' if you want to use GPU
        NUM_WORKERS = 0 

        

        train_dataset = loadDataSet(trainDF[cols],modelType)
        test_dataset = loadDataSet(testDF[cols],modelType)
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
        test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)


        lrate = 0.1

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters())

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)
        criterion.to(device)

        for epoch in range(NUM_EPOCHS):
            train_loss, train_accuracy = xtrain(model, device, train_loader, criterion, optimizer, epoch)
            torch.save(model, os.path.join(PATH_OUTPUT, save_file))

        best_model = torch.load(os.path.join(PATH_OUTPUT, save_file))
        test_loss, test_accuracy, test_results = evaluate(best_model, device, test_loader, criterion)
        output_dict[modelType]=pd.DataFrame(test_results,columns=['y_test','y_pred','y_prob'])
    return output_dict


## Model with out Notes

In [73]:
data = df.merge(dfNotes[['subject_id','cluster']])
features = [feature for feature in data.columns if feature not in 
            ['angus','subject_id','cluster','dob','gender','age_group','maxDate','first_admittime']]

In [74]:
train,test = train_test_split(data,test_size=0.2)



In [88]:
label = 'withOutNotes'

In [89]:
finalOutput[label] = dict()

In [90]:
output = pd.concat([output,runNewModel(train,test,Model(LogisticRegression(),'Logistic'))]) 
finalOutput[label]['Logistic'] = output
rocCurve(output,'Logistic with out notes')

//anaconda/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8059462436138065


//anaconda/envs/py36/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [85]:
output_dict = runCNNRNN(features+['angus'],'_with_Out_Notes',train,test,PATH_OUTPUT)

//anaconda/envs/py36/lib/python3.6/site-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type MyCNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
//anaconda/envs/py36/lib/python3.6/site-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type MyCNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
//anaconda/envs/py36/lib/python3.6/site-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type MyCNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
//anaconda/envs/py36/lib/python3.6/site-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type MyCNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checke

In [91]:
for modelType,output in output_dict.items():
    finalOutput[label][modelType] = output
    rocCurve(output,modelType)
    print(modelType)
    print("****************")

0.7794619818133321
CNN
****************
0.8175235858043551
RNN
****************


## Model with soft clustering

In [93]:
data_soft = df.merge(dfNotes)

In [94]:
features = [feature for feature in data_soft.columns if feature not in 
            ['angus','subject_id','cluster','dob','gender','age_group','maxDate','first_admittime']]

In [98]:
label = 'Notes_soft_Clustering'
train,test = train_test_split(data_soft,test_size=0.2)
finalOutput[label] = dict()

In [99]:

output = pd.concat([output,runNewModel(train,test,Model(LogisticRegression(),'Logistic'))]) 
finalOutput[label]['Logistic'] = output
rocCurve(output,'Logistic with out notes')

//anaconda/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8221343583273824


//anaconda/envs/py36/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [101]:
output_dict = runCNNRNN(features+['angus'],'_with_Notes',train,test,PATH_OUTPUT+'with_notes/')

//anaconda/envs/py36/lib/python3.6/site-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type MyCNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
//anaconda/envs/py36/lib/python3.6/site-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type MyCNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
//anaconda/envs/py36/lib/python3.6/site-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type MyCNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
//anaconda/envs/py36/lib/python3.6/site-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type MyCNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checke

In [102]:
for modelType,output in output_dict.items():
    finalOutput[label][modelType] = output
    rocCurve(output,modelType)
    print(modelType)
    print("****************")

0.875087887852916
CNN
****************
0.8222869401925891
RNN
****************


## Running model for each cluster

In [103]:
data = df.merge(dfNotes[['subject_id','cluster']])
features = [feature for feature in data.columns if feature not in 
            ['angus','subject_id','cluster','dob','gender','age_group','maxDate','first_admittime']]

In [105]:
label = 'KMeans'
train,test = train_test_split(data,test_size=0.2)
finalOutput[label] = dict()

In [106]:
output = pd.DataFrame()
for cluster in range(n_clusters):
    cluster_df = data.loc[data.cluster==cluster]
    train,test = train_test_split(cluster_df,test_size=0.2)
    output = pd.concat([output,runNewModel(train,test,Model(LogisticRegression(),'Logistic'))]) 

finalOutput[label]['Logistic'] = output
rocCurve(output,'Logistic_Kmeans')

//anaconda/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
//anaconda/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
//anaconda/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
//anaconda/envs/py36/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.798460183151687


In [107]:
output_dict = {'CNN':pd.DataFrame(),'RNN':pd.DataFrame()}
for cluster in range(n_clusters):
    cluster_df = data.loc[data.cluster==cluster]
    train,test = train_test_split(cluster_df,test_size=0.2)
    output_dict_cluster = runCNNRNN(features+['angus'],'_with_Out_Notes',train,test,PATH_OUTPUT+'cluster/')
    for modelType,output in output_dict_cluster.items():
        output_dict[modelType] = pd.concat([output_dict[modelType],output])

//anaconda/envs/py36/lib/python3.6/site-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type MyCNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
//anaconda/envs/py36/lib/python3.6/site-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type MyCNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
//anaconda/envs/py36/lib/python3.6/site-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type MyCNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
//anaconda/envs/py36/lib/python3.6/site-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type MyCNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checke

//anaconda/envs/py36/lib/python3.6/site-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type MyCNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
//anaconda/envs/py36/lib/python3.6/site-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type MyCNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
//anaconda/envs/py36/lib/python3.6/site-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type MyCNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
//anaconda/envs/py36/lib/python3.6/site-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type MyCNN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checke

In [108]:
for modelType,output in output_dict.items():
    finalOutput[label][modelType] = output
    rocCurve(output,modelType)
    print(modelType)
    print("****************")

0.8177081602502951
CNN
****************
0.7710027103582012
RNN
****************


## Saving All the Model results to a File

In [109]:
store = pd.HDFStore(PATH_OUTPUT+'all_output.h5')

In [110]:
for label,output_dict in finalOutput.items():
    for modelType,output in output_dict.items():
        store[label+'_'+modelType] = output

In [112]:
store.close()

## Output Analysis

In [116]:
def rocCurves(saveFig,PATH_OUTPUT):
    store = pd.HDFStore(PATH_OUTPUT+'all_output.h5')
    plt.figure(figsize=(12, 10))
    for key in store.keys():
        df = store[key]
        y_test,y_pred,y_prob = df.y_test,df.y_pred,df.y_prob
        model = key.replace("/","")
        logit_roc_auc = roc_auc_score(y_test, y_pred)
        fpr, tpr, thresholds = roc_curve(y_test, y_prob)
        plt.plot(fpr, tpr, label= model + ' (area = %0.2f)' % logit_roc_auc)
    store.close()
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")
    plt.savefig(PATH_OUTPUT+saveFig)
    plt.show()

In [117]:
rocCurves('analysis.jpg',PATH_OUTPUT)

## Confusion Matrix

In [138]:
from sklearn.metrics import confusion_matrix
import itertools
def plot_confusion_matrix1(cm, classes,PATH_OUTPUT,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.savefig(PATH_OUTPUT+title+'.jpg')
    plt.clf()

def plot_confusion_matrix(results, class_names,label,PATH_OUTPUT):
	# TODO: Make a confusion matrix plot.
	# TODO: You do not have to return the plots.
	# TODO: You can save plots as files by codes here or an interactive way according to your preference.
	# print(results)
	y_true,y_pred,_ = zip(*results)
	cnf_matrix = confusion_matrix(y_true, y_pred)
	plot_confusion_matrix1(cnf_matrix,class_names,PATH_OUTPUT,title=label)

In [139]:
store = pd.HDFStore(PATH_OUTPUT+'all_output.h5')
plt.figure(figsize=(7, 7))
for key in store.keys():
    df = store[key]
    y_test,y_pred,y_prob = df.y_test,df.y_pred,df.y_prob
    plot_confusion_matrix(zip(y_test,y_pred,y_prob),['No Sepsis','Sepsis'],key.replace("/",""),PATH_OUTPUT+'test/')
store.close()

## Accuray scores

In [132]:
accuracy = []
models = []
store = pd.HDFStore(PATH_OUTPUT+'all_output.h5')
for key in store.keys():
    df = store[key]
    y_test,y_pred,y_prob = df.y_test,df.y_pred,df.y_prob
    accuracy.append(accuracy_score(y_test,y_pred))
    models.append(key.replace("/",""))
store.close() 
accDF = pd.DataFrame({'model':models,'accuracy':accuracy})
accDF

,model,accuracy
0,KMeans_CNN,0.855069
1,KMeans_Logistic,0.857798
2,KMeans_RNN,0.846448
3,Notes_soft_Clustering_CNN,0.903078
4,Notes_soft_Clustering_Logistic,0.871207
5,Notes_soft_Clustering_RNN,0.866296
6,withOutNotes_CNN,0.856909
7,withOutNotes_Logistic,0.860183
8,withOutNotes_RNN,0.863130
